In [1]:
import torch as pt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pickle
from SymNet import SymNet
import h5py

In [ ]:
N_ngb, Nsites

## Make the symmetry-related parameters

In [29]:
GnnPerms = pt.tensor(GpermNNIdx).long()
NNsites = pt.tensor(NNsiteList)

Nlayers = 5
NchOuts = [16,16,16,16, 1]

Ng = GnnPerms.shape[0]
Ndim = 3
gdiags = pt.zeros(Ng*Ndim, Ng*Ndim).double()
for gInd, g in GIndtoGDict.items():
    rowStart = gInd * Ndim
    rowEnd = (gInd + 1) * Ndim
    gdiags[rowStart : rowEnd, rowStart : rowEnd] = pt.tensor(g.cartrot).double()

In [30]:
# Make shells for symmetry-grouping sites
shells = []
AllSites = set()
for siteInd in range(Nsites):
    if siteInd not in AllSites:
        Rsite = SiteIndtoR[siteInd]
        newShell = set()
        for gIng, g in GIndtoGDict.items():
            Rnew, _ = superBCC.crys.g_pos(g, Rsite, (0, 0))
            siteIndNew = RtoSiteInd[Rnew[0], Rnew[1], Rnew[2]]
            newShell.add(siteIndNew)
        AllSites.update(newShell)
        shells.append(list(newShell))

In [31]:
# Next, let's sort them according to distance
def sortkey(shell):
    R = SiteIndtoR[shell[0]]
    xmin = np.linalg.norm(np.dot(superBCC.crys.lattice, R))
    for siteInd in shell:
        R = SiteIndtoR[siteInd]
        x = np.linalg.norm(np.dot(superBCC.crys.lattice, R))
        if x < xmin:
            xmin = x
    return xmin
shellsSorted = sorted(shells, key=sortkey)

In [32]:
# Now let's assign shells to sites
SitesToShells = pt.zeros(Nsites).long()
for shellInd, shell in enumerate(shellsSorted):
    for siteInd in shell:
        SitesToShells[siteInd] = shellInd
pt.unique(SitesToShells, return_counts=True)

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
 tensor([ 1,  8,  6, 12, 24,  8,  6, 24, 24, 24,  8, 24, 12, 48, 24,  6, 24, 24,
         24,  4, 24, 24, 24, 24, 48,  3, 12, 12,  6]))

## Let's do backpropagation tests to make sure gradients are calculated correctly

In [33]:
# Let's take a dummy loss function that tries to force
# all vectors to -1
def loss_fn(rate, dx, dy):
    return pt.sum(rate * pt.norm(dx + dy)**2)/6.

In [34]:
TestNet = SymNet(Nlayers, NchOuts, GnnPerms, GIndtoGDict, gdiags, NNsites, SitesToShells,
                 Ndim, act="relu").double()

In [35]:
sm = 0.
for weight in TestNet.weightList:
    sm += weight.shape[0]*weight.shape[1]*weight.shape[2]
sm

7200.0

In [36]:
TestNet.ShellWeights.shape

torch.Size([29])

In [37]:
TestNet.wtVC.shape

torch.Size([3, 9])

In [38]:
sm = 0.
for bias in TestNet.biasList:
    sm += bias.shape[0]
sm

65.0

In [39]:
for bias in TestNet.biasList:
    print(bias.shape[0])

16
16
16
16
1


In [40]:
7200+29+24+65

7318